# Get weights from Fusion

In [1]:
cd /project2/mstephens/causalTWAS/fusion_weights

In [1]:
wget https://data.broadinstitute.org/alkesgroup/FUSION/WGT/YFS.BLOOD.RNAARR.tar.bz2

--2019-10-29 16:14:53--  https://data.broadinstitute.org/alkesgroup/FUSION/WGT/YFS.BLOOD.RNAARR.tar.bz2
Resolving data.broadinstitute.org (data.broadinstitute.org)... 69.173.92.29
Connecting to data.broadinstitute.org (data.broadinstitute.org)|69.173.92.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69800449 (67M) [application/x-bzip2]
Saving to: ‘YFS.BLOOD.RNAARR.tar.bz2’

100%[======================================>] 69,800,449  3.46MB/s   in 14s    

2019-10-29 16:15:08 (4.75 MB/s) - ‘YFS.BLOOD.RNAARR.tar.bz2’ saved [69800449/69800449]



In [3]:
tar xjf YFS.BLOOD.RNAARR.tar.bz2

In [2]:
ls

YFS.BLOOD.RNAARR       YFS.BLOOD.RNAARR.pos	 YFS.BLOOD.RNAARR.profile.err
YFS.BLOOD.RNAARR.list  YFS.BLOOD.RNAARR.profile  YFS.BLOOD.RNAARR.tar.bz2


# Process WTCCC data

WTCCC data obtained from Peter (`bd.RData`). We will prepare genotype file in BIMBAM format and relatedness matrix file.

In [3]:
# sinteractive --time=1-12:00:00 --partition=mstephens --mem=20G
module load R/3.5.1

In [15]:
cd /project2/mstephens/causalTWAS/causal-TWAS/code

In [33]:
Rscript run_WTCCC_data_process.R

In [17]:
ls /project2/mstephens/causalTWAS/WTCCC

bd.geno.txt.gz	bd.pheno.txt  bd.QC.RData  bd.RData


# Gene annotation data

In [11]:
mkdir /project2/mstephens/causalTWAS/annotation

In [21]:
cd /project2/mstephens/causalTWAS/annotation; wget http://hgdownload.soe.ucsc.edu/goldenPath/hg17/database/refGene.txt.gz

--2019-11-08 09:56:18--  http://hgdownload.soe.ucsc.edu/goldenPath/hg17/database/refGene.txt.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7550681 (7.2M) [application/x-gzip]
Saving to: ‘refGene.txt.gz’

100%[======================================>] 7,550,681   6.66MB/s   in 1.1s   

2019-11-08 09:56:19 (6.66 MB/s) - ‘refGene.txt.gz’ saved [7550681/7550681]



In [22]:
gunzip refGene.txt.gz

In [ ]:
Rscript /project2/mstephens/causalTWAS/causal-TWAS/code/gene_annotation.R refGene.txt 500000

In [24]:
ls

refGene_processed_5e+05.txt  refGene.txt


# UKbiobank data

## About CRI
Intructions to use CRI: https://github.com/stephenslab/gardner

* interactive job  
`qsub -I -l walltime=05:00:00 -l nodes=1:ppn=12 -l mem=32gb`
* submit a job  
`qsub ~/script/run_serverjob_template.sh`

## Data storage location
on CRI.
phenotype data: `/gpfs/data/xhe-lab/uk-biobank/`.  
genotype data:  `/gpfs/data/pierce-lab/uk-biobank-genotypes`.

## Prepare genotype file for chromosome 22, 20000 samples.

plink filtering instructions: https://www.cog-genomics.org/plink/2.0/filter

In [ ]:
# on CRI, the interactive node is very unstable and eventually killed due to memory isues
# prepare kept samples, take 20,000
cd /home/szhao1/uk-biobank/uk-biobank-genotypes; 
head -n 20002 ukb17346_imp_chr17_v3_s487378.sample | awk '{print $1"\t"$2}' > /home/szhao1/causal-TWAS/data/ukbiobank_samples20000.txt
# then manually remove the first two rows.
cd /home/szhao1/causal-TWAS/data; qsub qsub_prep_ukbiobank_n20000_chr22_genotype.sh

In [ ]:
# use sftp to download file to RCC (very fast)
cd /home/simingz/causalTWAS/ukbiobank
sftp szhao1@cri-syncmon.cri.uchicago.edu
cd /gpfs/data/pierce-lab/uk-biobank-genotypes
get ukb_imp_chr22_v3.bgen.*
get head -n 20002 ukb17346_imp_chr17_v3_s487378.sample

In [ ]:
head -n 20002 ukb17346_imp_chr17_v3_s487378.sample |  awk '{print $1"\t"$2}'> ukb17346_imp_chr17_v3_s20000.sample
# manually remove first two rows

**Process .bgen file using plink.**  
The original chr22 bgen file contains 1251k variants. After applied the maf (minor allele frequency) 5% cut off, 100k remains. then after applied missing rate 5% cut off (default is 10%), 80k remained.

Originall tried `--make-bed`, this resulted in missing genotype, so switched to `--make-pgen`.

> Some notes from Chirstopher Chang (author of plink)
1. You generally want to use PLINK 2.0 instead of 1.9 when working with .bgen files.  PLINK 1.9 is unable to track any of the genotype probability information in the .bgen; it rounds numbers to the nearest integer and replaces those too far from an integer with missing calls.
2. If you want to generate a .bgen file with only the variants you want, replace "--out rs6857.bgen" with "--export bgen-1.1 --out rs6857".  --out just specifies an output filename prefix; including ".bgen" in the --out parameter does not tell PLINK 2.0 to export a .bgen file.
3. This will still be a lossy process; PLINK 2.0 keeps track of dosages, but not genotype probabilities.  When sample x has P(AA) = 0.8, P(AC) = 0.2, and P(CC) = 0 and sample y has P(AA) = 0.85, P(AC) = 0.1, and P(CC) = 0.05, PLINK 2.0 sees both samples as dosage(A) = 1.8, dosage(C) = 0.2, and it will make both entries look like sample x when asked to export a .bgen.  Sometimes this is okay, but if you aren't sure, you should stick to qctool/bgenix for .bgen data management because of this information loss.

In [ ]:
# this step takes < 1 hour
sbatch submit_parallel_run_ukb_chr22_s20000.sh.sh